---

📁 **DocDefensor - Script de Geração de Documentos/Manifestações Automáticas [V.1.0.0]**  
🔗 Créditos: Desenvolvido por [João Victor](https://github.com/S1LV4/) | [joao.ai](https://linktr.ee/joao.ai)

---

### 🧭 **Como Usar: Passo a Passo**

#### 1️⃣ Faça **upload** dos seguintes arquivos na sessão de arquivos do **Google Colab**:

- `FORM_DOCUMENTO.docx` → Modelo do documento com as chaves definidas como `{{chaves}}`
- `documentos.csv` → Arquivo CSV com os valores correspondentes às chaves

> 💡 **Dica:** Caso deseje utilizar um modelo diferente, basta alterar os nomes dos arquivos conforme abaixo:

```python
CSV = BASE / "documentos.csv"
TEMPLATE = BASE / "FORM_DOCUMENTO.docx"
```

#### 2️⃣ Execute o processador com o seguinte comando:

```python
__procuracoes()
```

#### 3️⃣ Após a execução, os arquivos gerados estarão disponíveis para download na pasta:

```
/content/documentos/
```
---

In [11]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# Instalação de pacotes
import subprocess, sys
for p in ['python-docx', 'docxtpl', 'pandas']:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', p])

# Imports
import pandas as pd
from pathlib import Path
from datetime import datetime
from docxtpl import DocxTemplate
import re

In [12]:
# Caminhos
BASE = Path("/content")
CSV = BASE / "documentos.csv"
TEMPLATE = BASE / "FORM_DOCUMENTO.docx"

In [13]:
# Limpeza de nome para nome de arquivo
def __limpar_nome(nome):
    nome = str(nome).upper().strip()
    for a, s in {'Á':'A','À':'A','Ã':'A','Â':'A','É':'E','Ê':'E','Í':'I','Ó':'O','Ô':'O','Õ':'O','Ú':'U','Ç':'C'}.items():
        nome = nome.replace(a, s)
    nome = re.sub(r'[^A-Z0-9\s]', '', nome)
    return re.sub(r'\s+', '_', nome)[:40]

In [14]:
# Processamento principal
def __processar():
    if not CSV.exists():
        print("❌ CSV não encontrado.")
        return
    if not TEMPLATE.exists():
        print("❌ Template não encontrado.")
        return

    try:
        df = pd.read_csv(CSV)
    except Exception as e:
        print(f"❌ Erro ao ler CSV: {e}")
        return

    obrig = ['modelo','nome_outorgante','id_funcional_outorgante','cpf_outorgante',
             'nome_outorgado','identidade_outorgado','cpf_outorgado','data']
    if not all(col in df.columns for col in obrig):
        print(f"❌ Colunas faltando: {set(obrig) - set(df.columns)}")
        return

    pasta = BASE / "documentos" / datetime.now().strftime("%Y%m%d_%H%M%S")
    pasta.mkdir(parents=True, exist_ok=True)

    ok, erro = 0, 0
    for i, linha in df.iterrows():
        try:
            tpl_path = BASE / linha['modelo']
            if not tpl_path.exists():
                print(f"❌ Template não encontrado: {linha['modelo']}")
                erro += 1
                continue

            dados = {col: str(linha[col]).strip() for col in obrig if col != 'modelo'}
            nome_arq = f"DOC_{__limpar_nome(linha['nome_outorgante'])}.docx"
            doc = DocxTemplate(tpl_path)
            doc.render(dados)
            doc.save(pasta / nome_arq)
            ok += 1
        except Exception as e:
            print(f"❌ Erro na linha {i + 1}: {e}")
            erro += 1

    print(f"\n✅ {ok} geradas com sucesso | ❌ {erro} com erro(s)")
    print(f"📂 Arquivos em: {pasta}")


In [15]:
__processar()


✅ 9 geradas com sucesso | ❌ 0 com erro(s)
📂 Arquivos em: /content/documentos/20250729_201301
